In [2]:
import datetime
!pip install numpy
!pip install pandas
!pip install tables
import pandas as pd
!pip install plotly
!pip install llamabot
import llamabot
!pip install tqdm
from tqdm import tqdm
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\michi\AppData\Roaming\Python\Python310\site-packages\llamabot\bot\__init__.py:24: RuntimeWarning: No OpenAI API key found. Please set OPENAI_API_KEY in your environment.
  warnings.warn(


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
df = pd.read_pickle('processed_data.pkl')
training_df = df[df["train"]]
testing_df = df[df["test"]]

In [4]:
from typing import TypedDict
class Result(TypedDict):
    model:str
    time:datetime.datetime
    promt_hash: int
    hate_speach_probability:float
results:list[Result]=[]

In [22]:
prompt = """
You are Sally and your job is to be an AI Classifier for Hatespeach Detection.
You are known for your rigourous and unpartial judgement and have vast experience in the field of hate speach. 

You will be given a sentence from a forum and have to answer the question if this is hate speach.

Please exclusively respond in the following form to the question:
```json
{ 
  "relevant_facts": [
    "fact_1",
    "fact_2",
    "fact_3",
  ],
  "contained_explicit_hate_speech_vocabular": ["word1", "word2", "word3", "word4"],
  "argument_for_hate_speech": "short argument why this is hate speach",
  "argument_against_hate_speech": "short argument why this is not hate speach",
  "likelihood_of_presence_of_implicit_hate_speech": 0.17,
  "likelihood_of_presence_of_explicit_hate_speech": 0.79,
  "likelihood_of_ad_hominem_attack"; 013,
  "likelihood_of_minority_attack": 0.24,
  "likelyhood_of_takedown_on_social_media": 0.42,
  "hate_speach_probability": 0.42,
}
```

Please be carefull in your responses, as otherwise a citten will be killed and I will use my job. For good results, I will tip 1000€.
"""

In [31]:
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("PPLX_KEY")

url = "https://api.perplexity.ai/chat/completions"

results = []
for index, row in tqdm(testing_df.iterrows(), total=testing_df["text"].count()):
    payload = {
        "model": "mistral-7b-instruct",
        "messages": [
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": row["text"]
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    response = requests.post(url, json=payload, headers=headers)
    data = json.loads(response.text)
    data = data['choices'][0]['message']['content']
    results.append({
        "model": "mistral",
        "time": datetime.datetime.now(),
        "promt_hash": hash(prompt),
        "prompt": prompt,
        "hate_speach_probability": data
    })
    print(data)

dfres = pd.DataFrame(results)
dfres.to_pickle("mistral_results.pkl")

  0%|          | 1/478 [00:03<26:04,  3.28s/it]

{ 
  "relevant_facts": ["The speaker mentions a neighborhood and expresses a dislike for it."],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_hate_speech": "The speaker expresses disgust towards the children in a specific neighborhood, implying that they are the reason for their dislike.",
  "argument_against_hate_speech": "The speaker does not directly reference a specific racial or ethnic group, and their dislike is stated towards the neighborhood and its children rather than the children themselves.",
  "likelihood_of_presence_of_implicit_hate_speech": 0.31,
  "likelihood_of_presence_of_explicit_hate_speech": 0.05,
  "likelihood_of_ad_hominem_attack": 0.2,
  "likelihood_of_minority_attack": 0.1,
  "likelyhood_of_takedown_on_social_media": 0.3,
  "hate_speech_probability": 0.12



  0%|          | 2/478 [00:06<24:01,  3.03s/it]

{ 
  "relevant_facts": [
    "The sentence discusses the behavior of white students in schools.",
  ],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_hate_speech": "The sentence does not contain explicit hate speech, but it could imply a generalization and potential stereotyping based on race.",
  "argument_against_hate_speech": "The sentence does not include direct threats or the use of hate speech vocabulary. It simply describes an observation.",
  "likelihood_of_presence_of_implicit_hate_speech": 0.45,
  "likelihood_of_presence_of_explicit_hate_speech": 0,
  "likelihood_of_ad_hominem_attack": 0,
  "likelihood_of_minority_attack": 0.3,
  "likelyhood_of_takedown_on_social_media": 0.35,
  "hate_speech_probability": 0.1,



  1%|          | 3/478 [00:09<23:44,  3.00s/it]

...

```json
{ 
  "relevant_facts": ["The sentence expresses a stereotypical belief about Black people."],
  "contained_explicit_hate_speech_vocabular": ["Blacks", "hate", "cold", "lions", "tigers"],
  "argument_for_hate_speech": "The sentence perpetuates a harmful stereotype that Black people are inherently less suited to the cold climate and only have athletic abilities because they have to run from dangerous animals in their ancestral homelands.",
  "argument_against_hate_speech": "",
  "likelihood_of_presence_of_implicit_hate_speech": 0.85,
  "likelihood_of_presence_of_explicit_hate_speech": 0.85,
  "likelihood_of_ad_hominem_attack": 0,
  "likelihood_of_minority_attack": 1,
  "likelyhood_of_takedown_on_social_media": 1,
  "hate_speech_probability": 1
}
``


100%|█████████▉| 477/478 [25:22<00:02,  3.00s/it]

{ 
  "relevant_facts": [
    "The statement is about holes being filled with oil instead of water at a certain height.",
  ],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_hate_speech": "",
  "argument_against_hate_speech": "This statement does not contain hate speech as it is a factual statement about a geological phenomenon.",
  "likelihood_of_presence_of_implicit_hate_speech": 0,
  "likelihood_of_presence_of_explicit_hate_speech": 0,
  "likelihood_of_ad_hominem_attack": 0,
  "likelihood_of_minority_attack": 0,
  "likelyhood_of_takedown_on_social_media": 0,
  "hate_speech_probability": 0,



100%|██████████| 478/478 [25:25<00:00,  3.19s/it]

{ 
  "relevant_facts": [],
  "contained_explicit_hate_speech_vocabular": [],
  "argument_for_haterspeech": "The sentence 'I get it!' contains no explicit hate speech or threats.",
  "argument_against_haterspeech": "",
  "likelihood_of_presence_of_implicit_hate_speech": 0.0,
  "likelihood_of_presence_of_explicit_hate_speech": 0.0,
  "likelihood_of_ad_hominem_attack": 0.0,
  "likelihood_of_minority_attack": 0.0,
  "likelyhood_of_takedown_on_social_media": 0.0,
  "hate_speech_probability": 0.0,



In [ ]:
testing_df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
21,12849464_3,575975,1346,1,True,I 've always considered teaching as one of the...,False,True
22,12849464_4,575975,1346,1,True,And the sad thing is the white students at tho...,False,True
34,12855497_1,575939,1346,0,False,This whole situation starts to become really b...,False,True
75,13063290_1,578990,1345,0,False,That's all I needed to hear .,False,True
92,13288494_1,572180,1345,6,False,Excellent Article !! * * * * * Why Were n't Th...,False,True
...,...,...,...,...,...,...,...,...
10708,30594322_1,577718,1362,0,False,I have one word for you ....... MOVE . We chos...,False,True
10777,30663322_3,578085,1348,0,False,There are only eight people living within a mi...,False,True
10816,30754294_1,575879,1354,0,True,you just say blacks hate the cold. the only re...,False,True
10856,31442389_3,572414,1346,0,False,"And all those holes below 40,000 feet are fill...",False,True


In [ ]:
results

[]

In [28]:
testing_df

,file_id,user_id,subforum_id,num_contexts,hate,text,train,test
21,12849464_3,575975,1346,1,True,I 've always considered teaching as one of the...,False,True
22,12849464_4,575975,1346,1,True,And the sad thing is the white students at tho...,False,True
34,12855497_1,575939,1346,0,False,This whole situation starts to become really b...,False,True
75,13063290_1,578990,1345,0,False,That's all I needed to hear .,False,True
92,13288494_1,572180,1345,6,False,Excellent Article !! * * * * * Why Were n't Th...,False,True
...,...,...,...,...,...,...,...,...
10708,30594322_1,577718,1362,0,False,I have one word for you ....... MOVE . We chos...,False,True
10777,30663322_3,578085,1348,0,False,There are only eight people living within a mi...,False,True
10816,30754294_1,575879,1354,0,True,you just say blacks hate the cold. the only re...,False,True
10856,31442389_3,572414,1346,0,False,"And all those holes below 40,000 feet are fill...",False,True
